In [4]:
from src import Tensor,cross_entropy,Adam,SGD,Linear, Module, Sequential, ReLU,dataloader
import os
import sys
import numpy as np
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [5]:
import sklearn.datasets
train_X, train_y = sklearn.datasets.load_digits(return_X_y=True)

In [7]:
class Net(Module):
    def __init__(self, inc, outc):
        super().__init__()
        self.seq = Sequential(
            Linear(inc, 64, bias=True),
            ReLU(),
            Linear(64, 64*2, bias=True),
            ReLU(),
            Linear(64*2, outc),
        )

    def forward(self, x):
        x = self.seq(x)
        return x


net = Net(train_X.shape[1], len(np.unique(train_y)))
optimizer = Adam(net.trainable_params(), lr=0.001,amsgrad=True)
for epoch in range(10):  # loop over the dataset multiple times
    losses = []
    accs = []
    for X, y in dataloader(64*2, train_X, train_y):

        # forward + backward + optimize
        outputs = net.forward(X)
        loss = cross_entropy(outputs, y)

        optimizer.zero_grad()
        loss.backward()
        assert loss.item() > 0, loss
        optimizer.step()

        losses.append(loss.item())
        accs.append((outputs.data.argmax(-1) == y.data).astype(float).mean())
    print(
        f'[Epoch {epoch + 1:3}] loss: {np.mean(losses):.3f} accuracy: {np.mean(accs)*100:3.2f}')

print('Finished Training')

[Epoch   1] loss: 4.840 accuracy: 48.28
[Epoch   2] loss: 0.419 accuracy: 88.75
[Epoch   3] loss: 0.181 accuracy: 94.53
[Epoch   4] loss: 0.106 accuracy: 96.67
[Epoch   5] loss: 0.077 accuracy: 97.66
[Epoch   6] loss: 0.057 accuracy: 98.33
[Epoch   7] loss: 0.044 accuracy: 99.11
[Epoch   8] loss: 0.037 accuracy: 99.06
[Epoch   9] loss: 0.031 accuracy: 99.53
[Epoch  10] loss: 0.027 accuracy: 99.53
Finished Training


In [15]:
(train_y == net(Tensor(train_X)).data.argmax(-1)).mean()

0.9961046188091264